In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
import re
from cleantext import clean
import string

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:
df=pd.read_excel('Copy of Bot Data.xlsx')

#text preprocessing

#convert text to lower case 
df['biography']=df['biography'].str.lower()
df['comment']=df['comment'].str.lower()

#expand contractions 
contractions_dict = {"aren't": "are not", "don't": "do not", "Don't": "do not", "I'm": "I am", "i'm": "I am", 
                    "it's": "it is", "y'all": "you all", "Y'all": "you all", "didn't": "did not", "won't": "will not",
                   "I'll": "I will", "i'll": "I will", "can't": "can not"}
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))
def expand_contractions(text, expand_dict):
    def replace(match):
        return expand_dict[match.group(0)]
    return contractions_re.sub(replace, text)
df['biography']=df['biography'].apply(lambda x:expand_contractions(x, contractions_dict ))
df['comment']=df['comment'].apply(lambda x:expand_contractions(x, contractions_dict))
bios=df['biography'].astype(str)
usernames=df['Username']
comments=df['comment']
#filter out emojis 
new_bios=[]
for b in bios: 
    temp=clean(b, no_emoji=True)
    temp=temp.replace("\n", " ")
    if temp=='':
        new_bios.append('None')
    else: 
        new_bios.append(temp)
new_comments=[]
for c in comments:
    temp=clean(c, no_emoji=True)
    temp=temp.replace("\n", " ")
    if temp=='':
        new_comments.append('None')
    else: 
        new_comments.append(temp)
df_filtered=df.copy()
df_filtered['biography']=new_bios
df_filtered['comment']=new_comments
df_filtered['combined']=df_filtered['biography'] + ' ' + df_filtered['comment']
from sklearn.model_selection import train_test_split
#train and test set split 
train_set, test_set= train_test_split(
df_filtered, test_size=.2, random_state=10)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vector = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vector.fit(train_set['combined'])
tfidf_array =  tfidf_vector.transform(train_set['combined']).toarray()
text_df = pd.DataFrame(data=tfidf_array,columns = tfidf_vector.get_feature_names())
text_df

,0,000,0381,05,0598,064acf,09,1,10,100,...,youtu,youtube,yt,zainab,zainabzilahi,zeldon,zero,zilahi,zooming,zu
0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.205384,0.0,0.0,0.152454,0.156599,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
247,0.0,0.152785,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
248,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
249,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

model = LogisticRegression(max_iter=1000)
selector1=RFE(estimator=model, n_features_to_select=20, step=1) #chose arbitrarily to select 20 of the most important words out of 2000 to make it more manageable

#split dataset into features and labels
Y1=train_set['bot classification (0-not a bot, 1-bot)']
X1=text_df

#run RFE
rfe1 = selector1.fit_transform(X1, Y1)
filter1=(selector1.get_support())

##
#filter columns using data from RFE
filter1=list(filter1)
current_cols = list(X1.columns)

#figure out which columns to keep
important_cols=[]
for index in range(len(filter1)):
    if (filter1[index])==True:
        important_cols.append(current_cols[index])
important_cols

['5',
 'app',
 'bio',
 'click',
 'content',
 'dm',
 'do',
 'first',
 'i',
 'link',
 'linkr',
 'ly',
 'me',
 'my',
 'netlify',
 'people',
 'that',
 'the',
 'to',
 'you']

In [5]:
train_set_filtered=train_set[['Follower Count', 'Following Count',
       'Follower/Following Ratio', 'Number of posts', '# of likes on comment ', 'time of comment after post (minutes)', 'bot classification (0-not a bot, 1-bot)']]
df_normalized=train_set_filtered.copy()
numerical_cols=['Follower Count', 'Following Count',
       'Follower/Following Ratio', 'Number of posts', '# of likes on comment ', 'time of comment after post (minutes)']
for c in numerical_cols:
    df_normalized[c]= (train_set_filtered[c] - train_set_filtered[c].min()) / (train_set_filtered[c].max() - train_set_filtered[c].min())
df_normalized=df_normalized.reset_index(drop=True)

#normalize the test set 
test_normalized=test_set.copy()
test_normalized=test_normalized[['Follower Count', 'Following Count',
       'Follower/Following Ratio', 'Number of posts', '# of likes on comment ', 'time of comment after post (minutes)', 'bot classification (0-not a bot, 1-bot)']]
for c in numerical_cols: 
    test_normalized[c]= (test_normalized[c] - test_normalized[c].min()) / (test_normalized[c].max() - test_normalized[c].min())  
test_normalized=test_normalized.reset_index(drop=True)

#convert the text columns to arrays and merge dataframes 
text_array_test = tfidf_vector.transform(test_set['combined']).toarray()
text_df_test = pd.DataFrame(data=text_array_test,columns = tfidf_vector.get_feature_names())



In [6]:
features_filtered=X1[important_cols]
#merge with original numerical columns
features_filtered=pd.concat([features_filtered, df_normalized], axis=1)
X_train=features_filtered.drop(['bot classification (0-not a bot, 1-bot)'], axis=1)
Y_train=features_filtered['bot classification (0-not a bot, 1-bot)']

#filter test set for important columns 
text_df_test=text_df_test[important_cols]
df_combined_test=pd.concat([text_df_test, test_normalized], axis=1)
df_combined_test.replace([np.inf, -np.inf], np.nan, inplace=True)
df_combined_test=df_combined_test.dropna()
X_test=df_combined_test.drop(['bot classification (0-not a bot, 1-bot)'], axis=1)
Y_test=df_combined_test['bot classification (0-not a bot, 1-bot)']

#logistic regression 
classifier = LogisticRegression()
classifier.fit(X_train, Y_train)
score = classifier.score(X_test, Y_test)
print("Accuracy:", score)

Accuracy: 0.9047619047619048


In [7]:
#K-fold cross validation again 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
LR = LogisticRegression(max_iter=1000)
cv = KFold(n_splits=10, random_state=0, shuffle=True)
result = cross_val_score(LR, X_train, Y_train, cv=cv, scoring='accuracy')
print('Accuracy: %.3f (%.3f)' % (mean(result), std(result)))


Accuracy: 0.948 (0.059)


In [8]:
#hyperparameter tuning 
from sklearn.model_selection import GridSearchCV
h_model=LogisticRegression(max_iter=1000)
grid = dict()
grid['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
grid['penalty'] = ['l1', 'l2', 'none']
grid['C'] = np.logspace(-4, 4, 10) #range of values to test
grid_search = GridSearchCV(h_model, grid, cv = 5, scoring = 'accuracy',n_jobs=-1, verbose=True)
final= grid_search.fit(X_train, Y_train)
print('Best Score: %s' % final.best_score_)
print('Best Hyperparameters: %s' % final.best_params_)
final_model = final.best_estimator_

Fitting 5 folds for each of 90 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.7s


Best Score: 0.9561568627450981
Best Hyperparameters: {'C': 21.54434690031882, 'penalty': 'l1', 'solver': 'liblinear'}


[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:    3.0s finished


In [9]:
#looking at feature importance
importance = final_model.coef_[0]
indices = (-importance).argsort()[:5]
col_names=X_test.columns.tolist()
important_features = [col_names[i] for i in indices]
important_features

['do', 'dm', 'ly', 'content', 'people']